<a href="https://colab.research.google.com/github/smf-9000/kaggle/blob/main/%5B003%5D_tabular_playground_series_aug_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from statsmodels.nonparametric.smoothers_lowess import lowess
import warnings

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2021/train.csv
/kaggle/input/tabular-playground-series-aug-2021/test.csv


In [ ]:
submission_ex = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
targets_df = train_df[['loss']].copy()
train_df.drop(['loss', 'id'], axis=1, inplace=True) 

In [ ]:
for df in [train_df, test_df]:
    df['f27_0'] = ((df['f27'].values <= 50000)).astype(int)
    df['f27_1'] = ((df['f27'].values > 50000)).astype(int)
    df['f86_0'] = ((df['f86'].values < 200)).astype(int)
    c = (df['f86'] >= 200) & (df['f86'] < 450)
    df['f86_1'] = np.where(c, 1, 0)
    c = (df['f86'] >= 450) & (df['f86'] < 800)
    df['f86_2'] = np.where(c, 1, 0)
    df['f86_3'] = ((df['f86'].values >= 800)).astype(int)

In [ ]:
# from sklearn import preprocessing
# # scaler = preprocessing.MinMaxScaler()
# # scaler = preprocessing.MaxAbsScaler()
# scaler = preprocessing.StandardScaler()
# for df in [train_df, test_df]:
#     for c in df.columns:
#         if c not in ['id']:
#             df[[c]] = scaler.fit_transform(df[[c]])

In [ ]:
# for df in [train_df, test_df]:
#     for c in df.columns:
#         if c not in ['id']:
#             a = df[c].values
#             n = len(a)
#             for i in range(len(a)):
#                 if i == 0:
#                     a[i] = (a[i] + a[i+1]) / 2
#                 elif i == n-1:
#                     a[i] = (a[i-1] + a[i]) / 2
#                 else:
#                     a[i] = (a[i-1] + a[i] + a[i+1]) / 3
#             df[c+'_s'] = a

In [ ]:
# warnings.filterwarnings("ignore")

# # ################################################################
# fig = plt.figure(figsize = (30,30))
# ax = fig.gca()
# hist = train_df.hist(bins=50, ax = ax)

In [ ]:
# for c in train_df.columns:
#     if c != 'id':
#         fig = plt.figure(figsize = (30, 3))
#         g = train_df.iloc[:500,:][c].plot()
#         g = targets_df.iloc[:500,:]['loss'].plot()
#         g.legend([c])

In [ ]:
# train_df.iloc[0:10, 0:50].info()

In [ ]:
# train_df.iloc[0:10, 27]

In [ ]:
# warnings.filterwarnings("ignore")

# # ################################################################
# fig = plt.figure(figsize = (30,10))
# ax = fig.gca()
# hist = targets_df[targets_df.columns.values].hist(bins=84, ax = ax)

In [ ]:
# warnings.filterwarnings("ignore")

# train_without_zero_loss_df = train_df[targets_df['loss'] != 0]

# # ################################################################
# fig, axes = plt.subplots(nrows = 10, ncols = 10, figsize=(30, 30))
# i = 0
# for c in train_df.columns[1:]:
#     ax = axes[i // 10, i % 10]
#     ax.set_title(c)
#     train_df[c].hist(bins=50, ax = ax)
#     train_without_zero_loss_df[c].hist(bins=50, ax = ax, alpha = 0.7)
#     i += 1

### feature correlation

In [ ]:
# qq_df = pd.concat([train_df, targets_df], axis=1, join='inner')

# # ################################################################
# corr = qq_df.corr()

# mask = np.zeros_like(corr)
# mask[np.triu_indices_from(mask)] = True

# fig = plt.figure(figsize = (25,25))
# sns.heatmap(corr, cmap="flare", mask=mask)
# plt.show()

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import warnings
from sklearn.model_selection import KFold

In [ ]:
def bayesion_opt_lgbm(X, y, init_points = 3, n_iter = 7, random_state = 42, seed = 42, _folds = 5, bo_verbose = 1, shuffle = False):
    dtrain = lgb.Dataset(data = X, label = y)
    print('no_of_folds: ', _folds)
    print('random_state: ', random_state)
    folds = KFold(_folds, shuffle = shuffle)

    def rmse_score(preds, dtrain):
        labels = dtrain.get_label()
        tmp = np.array(preds)
        return 'rmse', -mean_squared_error(labels, preds, squared = False), True

    def hyp_lgbm(learning_rate, 
                num_leaves, 
                max_depth, 
                n_estimators,
                min_sum_hessian_in_leaf,
                lambda_l1,
                lambda_l2,
                bagging_fraction,
                bagging_freq):

        params = {'application':'regression', 'metric':'rmse_score', 'force_col_wise':True, 'verbose': -1}
        params['learning_rate'] = learning_rate
        params['num_leaves'] = int(round(num_leaves))
        params['max_depth'] = int(round(max_depth))
        params['n_estimators'] = int(round(n_estimators))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['lambda_l1'] = 10 ** lambda_l1
        params['lambda_l2'] = 10 ** lambda_l2
        params['bagging_fraction'] = bagging_fraction
        params['bagging_freq'] = int(round(bagging_freq))
        
        cv_results = lgb.cv(
            params, 
            dtrain, 
            folds=folds.split(X), 
#             nfold = _folds, 
            seed = random_state, 
            stratified = False, 
            verbose_eval = None, 
            feval = rmse_score)

        return np.max(cv_results['rmse-mean'])
    pds = {
        'learning_rate': (0.001, 0.1),
        'num_leaves': (2, 256),
        'max_depth': (-1, 100),
        'n_estimators': (100, 100),
        'min_sum_hessian_in_leaf': (1e-8, 1000),
        'lambda_l1': (-5, 1),
        'lambda_l2': (-5, 1),
        'bagging_fraction': (0.4, 1.0),
        'bagging_freq': (1, 7),
        }

    optimizer = BayesianOptimization(hyp_lgbm, pds, random_state = random_state, verbose = bo_verbose)

    # n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    # init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    optimizer.maximize(init_points = init_points, n_iter = n_iter)

    return optimizer.max

In [ ]:
# warnings.filterwarnings("ignore")

# X, y = None, None

# lgbm = lgb.LGBMRegressor()
# _target = 'loss'
# X = np.array(train_df, dtype='float32')[:250000]
# y = np.squeeze(np.array(targets_df[[_target]].values, dtype='float32'))[:250000]
# print('X_shape: ', X.shape)

# data = bayesion_opt_lgbm(X, y, init_points = 10, n_iter = 20, _folds = 10, bo_verbose = 2, random_state = None, shuffle = True)  #init_points=random, n_iter=bayesian
# for k, v in data['params'].items():
#     if k in ['bagging_freq', 'max_depth', 'n_estimators', 'num_leaves']:
#         data['params'][k] = int(round(v))
#     elif k in ['lambda_l1', 'lambda_l2']:
#         data['params'][k] = 10 ** v
# print('params_' + str(_target) + ' =', data['params'])

In [ ]:
params_loss = {'bagging_fraction': 0.8962250586874196, 'bagging_freq': 1, 'lambda_l1': 0.01895614209401383, 'lambda_l2': 0.001397920954070796, 'learning_rate': 0.05906516799338329, 'max_depth': 46, 'min_sum_hessian_in_leaf': 376.9636549547788, 'n_estimators': 100, 'num_leaves': 184}
# params_loss = {'bagging_fraction': 0.9962738592807339, 'bagging_freq': 5, 'lambda_l1': 0.0007443736962799421, 'lambda_l2': 0.0006596033786078184, 'learning_rate': 0.08281394225689416, 'max_depth': 36, 'min_sum_hessian_in_leaf': 513.7609568818598, 'n_estimators': 100, 'num_leaves': 54}



params_loss['n_estimators'] = 10000
params_loss['learning_rate'] = 0.001

params_loss['n_estimators'] = 20000
lgbm_reg = lgb.LGBMRegressor(**params_loss, 
                       objective='rmse',
                       metric='rmse',
                       n_jobs=-1)

In [ ]:
submit = True

In [ ]:
submission = test_df[['id']].copy()

_target = 'loss'
X = np.array(train_df, dtype='float32')
y = np.squeeze(np.array(targets_df[[_target]].values, dtype='float32'))
X_test = np.array(test_df.iloc[:,1:].values, dtype='float32')

if not submit:
    split_train = 225000
    X, X_val = X[:split_train], X[split_train:]
    y, y_val = y[:split_train], y[split_train:]
    
    lgbm_reg.fit(
        X, 
        y,
        eval_set=[(X_val, y_val)],
        verbose=100,
        early_stopping_rounds=500)
    p1 = lgbm_reg.predict(X_val)

    print('target: ', _target)
    print('=================================')
    print('rmse: ', mean_squared_error(y_val, p1, squared = False))
else:
    print('submit: ', _target)

    lgbm_reg.fit(X, y)
    print('prediction')
    p1 = lgbm_reg.predict(X_test)

    submission[_target] = p1

submit:  loss
[LightGBM] [Warning] lambda_l1 is set=0.01895614209401383, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01895614209401383
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=376.9636549547788, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=376.9636549547788
[LightGBM] [Warning] lambda_l2 is set=0.001397920954070796, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.001397920954070796
[LightGBM] [Warning] bagging_fraction is set=0.8962250586874196, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8962250586874196
prediction


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
test = pd.read_csv('submission.csv')
test.head()

,id,loss
0,250000,7.402576
1,250001,5.086758
2,250002,8.778332
3,250003,7.893456
4,250004,7.180392


In [ ]:
test.tail()

,id,loss
149995,399995,8.136704
149996,399996,7.936379
149997,399997,6.112471
149998,399998,5.569623
149999,399999,6.342379
